# Nonlinear Solver

Many problems in finance (bond yield calculation, implied volatility calculation) require solving a nonlinear equation. In this section I explored the following 3 basic nonlinear solvers that can be the starting point and foundation for more complex solvers:
* **Bisection Method**
* **Newtown Method**
* **Secant Method**

Application of these Nonlinear solvers in the folllowing areas are also discussed:
* **Yield Caculation for a bond**
* **Implied Volatility Calculation for an option**
* **Spot Rate Curve Calculation from bond prices**

In [1]:
import numpy as np
from math import exp
%precision 6

'%.6f'

## Nonlinear Methods

In [2]:
# Evaluation function
def func(x):
    return x**4 - 5*x**2 + 4 - 1/(1+exp(x**3))

** Bisection Method**

Bisection method only works under the case that f is a continous function with different sign on the chosen interval

In [3]:
def Bisection(a,b,tol_int):
    # a: left endpoint of the interval
    # b: right endpoint of the terval
    # tol_int: largest admissible size of active interval when solution is found
    xl = a 
    xr = b
    itera = 0
    if func(xl)==0:
        return xl
    elif func(xr)==0:
        return xr
    elif func(xl)*func(xr)>0:
        print('Error: No solution satisfied Bisection method')
        return None
    else:
        mid = (xl+xr)/2
        error=(xr-xl)/2
        itera = 0
        while(error>tol_int):
            if func(mid)==0:
                return mid
            elif func(xr)*func(mid)<0:
                xl = mid
            elif func(xl)*func(mid)<0:
                xr = mid
            mid = (xl + xr)/2
            error = (xr-xl)/2
            itera = itera+1
        return mid,itera
                
    

In [4]:
# It is shown that with a error tolerance of 0.000000001, after 32 iterations, we got the solution  -0.889642
Bisection(-2,3,0.000000001)

(-0.889642, 32)

** Newtown's Method **

One of the most commonly used nonlinear solve is Newtown's method. It requires that the function is differentiable. Also, if we start from a point close enough to the solution, Newtown's method would converge quadratically. However, it requires that we have the differential function analytically.

In [5]:
# First we need to define the differential fuction
def func_d(x):
    return 4*x**3-10*x+3*x**2*exp(x**3)/(1+exp(x**3))**2

In [6]:
def Newtown(x_0, tol_int):
    x_new = x_0
    if func(x_0) == 0:
        return x_0
    else:
        while abs(func(x_new))>tol_int:
            x_old = x_new
            x_new = x_old - func(x_old)/func_d(x_old)
    return x_new            

In [7]:
## It should be noted that the starting point matters for our final solution
% time print(Newtown(3,0.000000001))
print(func(Newtown(3,0.000000001)))

2.0000279352906616
CPU times: user 80 µs, sys: 0 ns, total: 80 µs
Wall time: 83.9 µs
5.471666629985536e-10


In [8]:
% time print(Newtown(2,0.000000001))
print(func(Newtown(2,0.000000001)))

2.000027935245084
CPU times: user 64 µs, sys: 0 ns, total: 64 µs
Wall time: 67.9 µs
4.016318527755303e-15


In [9]:
% time print(Newtown(1,0.000000001))
print(func(Newtown(1,0.000000001)))

0.9507482560164099
CPU times: user 103 µs, sys: 1e+03 ns, total: 104 µs
Wall time: 108 µs
2.7755575615628914e-16


The time and iteration it takes to reach a solution also differs on the starting point. Now let's take a look at the convergent speed.

In [10]:
# Starting at 3
x = []
x.append(3) 
while abs(func(x[-1]))>0.000000001:
    x_new = x[-1] - func(x[-1])/func_d(x[-1])
    x.append(x_new)
x = np.array(x)
error = np.abs(x-2.000028)
print(x)
print(error)


[ 3.        2.487179  2.178029  2.035583  2.001875  2.000033  2.000028]
[  9.999720e-01   4.871515e-01   1.780007e-01   3.555470e-02   1.846824e-03
   5.304701e-06   6.470934e-08]


In [11]:
# Starting at 5
x = []
x.append(5) 
while abs(func(x[-1]))>0.000000001:
    x_new = x[-1] - func(x[-1])/func_d(x[-1])
    x.append(x_new)
x = np.array(x)
error = np.abs(x-2.000028)
print(x)
print(error)

[ 5.        3.88      3.08263   2.54078   2.207541  2.046147  2.003067
  2.000042  2.000028  2.000028]
[  2.999972e+00   1.879972e+00   1.082602e+00   5.407516e-01   2.075132e-01
   4.611911e-02   3.039043e-03   1.443468e-05   6.442257e-08   6.475492e-08]


** Secant Method **

This is the an approximation for Newtown method when an analytical differential function doesn't exist.

In [12]:
def Secant(x_s,x_0,tol_int):
    x_new = x_0
    x_old = x_s
    while abs(func(x_new))>tol_int:
        x_oldest = x_old
        x_old = x_new
        x_new = x_old - func(x_old)*(x_old-x_oldest)/(func(x_old)-func(x_oldest))
    return x_new

In [13]:
Secant(-0.01,3,0.00001)

-0.889642